In [ ]:
import pandas as pd
import numpy as np
import pymongo                                                  
from dotenv import load_dotenv
import os
import time
from datetime import datetime
import nba_api_functions_hist as hist
import nba_prop_functions_hist as prop_hist
import random
import uuid
import threading
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

In [ ]:
# ***** SHOULD UPDATE TO BE A LITTLE MORE RESTRICTIVE*********
        # mins OR individual stat at about that level

# limiting the df to players who hit at least one of these criteria
# all set ~180th in the league
min_min = hist_df['min_season_avg'] >= 19
pts_min = hist_df['pts_season_avg'] >= 8
reb_min = hist_df['reb_season_avg'] >= 3
ast_min = hist_df['ast_season_avg'] >= 1.5
stl_min = hist_df['stl_season_avg'] >= .5
blk_min = hist_df['blk_season_avg'] >= .25
fg3m_min = hist_df['fg3m_season_avg'] >= .7

filter_mask = min_min | pts_min | ast_min | reb_min | stl_min | blk_min | fg3m_min 

hist_df = hist_df[filter_mask]

In [ ]:
# ************NEED TO CHANGE THIS TO SMALLER INCREMENTS, BOTH BY HALF POINT AND BY X%*********
# creating new columns for each of the prop lines to test
prop_names = ['pts','reb','ast','stl','blk', 'fg3m']
prop_columns = []
for prop in prop_names:
    down_50 = prop+'_down_50'
    down_33 = prop+'_down_33'
    down_16 = prop+'_down_16'
    up_17 = prop+'_up_17'
    up_34 = prop+'_up_34'
    up_50 = prop+'_up_50'
    new_columns = [down_50, down_33, down_16, up_17, up_34, up_50]
    prop_columns.extend(new_columns)

prop_cols_dict = {col: None for col in prop_columns}

hist_df = hist_df.assign(**prop_cols_dict)

def create_prop_lines(df, prop):
    for index, row in df.iterrows():
        df.loc[index, prop+'_down_50'] = row[prop+'_season_avg'] * .5
        df.loc[index, prop+'_down_33'] = row[prop+'_season_avg'] * .67
        df.loc[index, prop+'_down_16'] = row[prop+'_season_avg'] * .84
        df.loc[index, prop+'_up_17'] = row[prop+'_season_avg'] * 1.17
        df.loc[index, prop+'_up_34'] = row[prop+'_season_avg'] * 1.34
        df.loc[index, prop+'_up_50'] = row[prop+'_season_avg'] * 1.5

for prop in prop_names:
    create_prop_lines(hist_df, prop)

# rounding columns to decrease size of df
cols_round = [
    'min_season_avg','pts_season_avg', 'reb_season_avg', 'ast_season_avg', 
    'stl_season_avg','blk_season_avg', 'fg3m_season_avg', 'pts_down_50', 
    'pts_down_33','pts_down_16', 'pts_up_17', 'pts_up_34', 'pts_up_50', 
    'reb_down_50','reb_down_33', 'reb_down_16', 'reb_up_17', 'reb_up_34', 
    'reb_up_50','ast_down_50', 'ast_down_33', 'ast_down_16', 'ast_up_17', 
    'ast_up_34','ast_up_50', 'stl_down_50', 'stl_down_33', 'stl_down_16', 
    'stl_up_17','stl_up_34', 'stl_up_50', 'blk_down_50', 'blk_down_33', 
    'blk_down_16','blk_up_17', 'blk_up_34', 'blk_up_50', 'fg3m_down_50', 
    'fg3m_down_33','fg3m_down_16', 'fg3m_up_17', 'fg3m_up_34', 'fg3m_up_50'
]

hist_df[cols_round] = hist_df[cols_round].astype('float64')
hist_df[cols_round] = hist_df[cols_round].round(1)

In [ ]:
columns = ['pts_down_50', 'pts_down_33','pts_down_16','pts_season_avg', 'pts_up_17', 'pts_up_34', 'pts_up_50']
prop = 'pts'
results_list = []
for column in columns:
    results_df = prop_hist.generate_test_prop_results_test(hist_df, column)
    target_df = hist_df[['unique_id',prop, column, f'{prop}_season_avg']]
    target_df['hit_over'] =  target_df.apply(lambda row: 1 if row[prop] > row[column] else 0, axis=1)
    results_df = pd.merge(results_df, target_df, on='unique_id')
    results_list.append(results_df)
    end_time = time.time()
    col_end = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(end_time))
    print(column + ' done at ' + col_end)

pts_results_df = pd.concat(results_list, axis=0)